In [ ]:
##reference: https://pyscenic.readthedocs.io/en/latest
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix

import json
import base64
import zlib
from pyscenic.plotting import plot_binarization
from pyscenic.export import add_scenic_metadata
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.set_figure_params(dpi=300, fontsize=10, dpi_save=600)

# Set maximum number of jobs for Scanpy.
sc.settings.njobs = 40

In [ ]:
sample_SCENIC = '//////.loom'

In [ ]:
lf = lp.connect(sample_SCENIC, mode='r+', validate=False )
auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()
auc_mtx

In [ ]:
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
from adjustText import adjust_text
from pyscenic.binarization import binarize

In [ ]:
sce_test = sc.read_h5ad("//////.h5ad")

In [ ]:
#Calculate RSS
rss_cellType = regulon_specificity_scores( auc_mtx, sce_test.obs.celltypenewaddsub )
rss_cellType

In [ ]:
sce_test.obs['celltypenewaddsub'].unique()

In [ ]:
add_scenic_metadata(sce_test, auc_mtx)

In [ ]:
#Calculate Z-score
rss_cellType_Z = pd.DataFrame( index=rss_cellType.index )
for col in list(rss_cellType.columns):
    rss_cellType_Z[col] = ( rss_cellType[col] - rss_cellType[col].mean()) / rss_cellType[col].std(ddof=0)
    rss_cellType_Z.sort_index(inplace=True)

rss_cellType_Z

In [ ]:
#stromal subpopulations
sce_sub = sce_test[sce_test.obs['celltypenewaddsub'].isin(['TXN+ stro','IGFBP3+ stro','CNN1+ stro'])]
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
sce_sub.obs['celltypenewaddsub'].unique()

In [ ]:
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
#按照分组计算rss
rss_group = regulon_specificity_scores( rss_epi, sce_sub.obs['celltypenewaddsub'])
rss_group

In [ ]:
rss_group.to_csv("//////.csv",index=True,sep=',')

In [ ]:
rss_group

In [ ]:
#heatmap
colors = ['#C6DBEFFF','#6BAED6FF','#08519CFF']
tf=['GATA6(+)','TBX1(+)','GATA4(+)','GATA1(+)','POU6F1(+)','GLI1(+)','ELK3(+)','IRF8(+)','OSR2(+)','SP3(+)','EGR2(+)','PAX2(+)','TBX15(+)','ZBTB6(+)','TBX3(+)']
sns.set(font_scale=1.2)
g = sns.clustermap(rss_group[tf],z_score=1, 
                   annot=False,  square=False,  cmap="vlag",
                   linecolor='black',
                   yticklabels=True, xticklabels=True, row_cluster=False,col_cluster=False, 
                   vmin=-0.7, vmax=0.7, 
                   row_colors=colors, figsize=(8,4) )
g.cax.set_visible(True)
g.ax_heatmap.set_ylabel('stromal subcluster')
g.ax_heatmap.set_xlabel('')

plt.savefig("///////SCENIC_stromalsubclusters.pdf")

In [ ]:
sce_test.obs['celltypenewaddsub'].unique()

In [ ]:
#CNN1+ stromal
sce_sub = sce_test[sce_test.obs['celltypenewaddsub'].isin(['CNN1+ stro'])]
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
sce_sub.obs['celltypenewaddsub'].unique()

In [ ]:
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
rss_group = regulon_specificity_scores( rss_epi, sce_sub.obs['slice'])
rss_group

In [ ]:
rss_group.to_csv("///////AM_ST_CNN1stro_diff_slice.csv",index=True,sep=',')

In [ ]:
rss_group

In [ ]:
#heatmap
colors = ['#92C5DEFF','#FDDBC7FF', '#F4A582FF','#D6604DFF']
tf=['TCF21(+)','ELK1(+)','CREB3(+)','GATA6(+)','ZFP3(+)','ZNF112(+)','MYCN(+)','ZNF880(+)','ZNF440(+)','FOXA2(+)','CREB3L4(+)','DLX5(+)','PAX2(+)','CREB3L1(+)']
sns.set(font_scale=1.2)
g = sns.clustermap(rss_group[tf],z_score=1, 
                   annot=False,  square=False,  
                   linecolor='black',
                   yticklabels=True, xticklabels=True, row_cluster=False,col_cluster=False,
                   vmin=-0.5, vmax=0.5, 
                   row_colors=colors, cmap="vlag", figsize=(8,4) )
g.cax.set_visible(True)
g.ax_heatmap.set_ylabel('CNN1stro')
g.ax_heatmap.set_xlabel('')

plt.savefig("///////SCENIC_CNN1stro_diff_slice.pdf")

In [ ]:
sce_test.obs['celltypenewaddsub'].unique()

In [ ]:
#ESR1+SMC
sce_sub = sce_test[sce_test.obs['celltypenewaddsub'].isin([ 'ESR1+ SMC'])]
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
rss_epi=auc_mtx.T[sce_sub.obs.index]
rss_epi = rss_epi.T
rss_epi

In [ ]:
sce_sub.obs['celltypenewaddsub'].unique()

In [ ]:
sce_sub.obs['slice'].unique()

In [ ]:
rss_group = regulon_specificity_scores( rss_epi, sce_sub.obs['slice'])
rss_group

In [ ]:
rss_group.to_csv("//////AM_ST_ESR1SMC_diff_slice.csv",index=True,sep=',')

In [ ]:
#heatmap
colors = [ '#92C5DEFF','#FDDBC7FF', '#F4A582FF','#D6604DFF']
tf=['TCF21(+)','ELK1(+)','TAGLN2(+)','USF2(+)','MAZ(+)','YBX1(+)','E2F4(+)','HDAC2(+)','ESR1(+)','DMRTA2(+)','SOX17(+)','EGR3(+)','CREB3L4(+)','HMX1(+)','PAX2(+)','ZNF567(+)']
sns.set(font_scale=1.2)
g = sns.clustermap(rss_group[tf],z_score=1, 
                   annot=False,  square=False,  
                   linecolor='black',
                   yticklabels=True, xticklabels=True, row_cluster=False, col_cluster=False,
                   vmin=-0.5, vmax=0.5, 
                   row_colors=colors,cmap="vlag", figsize=(8,4) )
g.cax.set_visible(True)
g.ax_heatmap.set_ylabel('ESR1SMC')
g.ax_heatmap.set_xlabel('')

plt.savefig("//////SCENICESR1SMC_diffslice.pdf")